In [22]:
import mysql.connector
import pandas as pd
import re
from bs4 import BeautifulSoup

In [ ]:
# Connect to the MySQL database
#replace connector information
conn = mysql.connector.connect(
    host="database.example.com",       
    user="user",   
    password="P@ssord", 
    database="database name"  
)

# Create a cursor object
cursor = conn.cursor()

# Execute a sample query
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

# Print the results
print("Tables in the database:")
for table in tables:
    print(table)

Tables in the database:
('auth_group',)
('auth_group_permissions',)
('auth_permission',)
('auth_user',)
('auth_user_groups',)
('auth_user_user_permissions',)
('chatbot_be_answer',)
('chatbot_be_conversation',)
('chatbot_be_document',)
('chatbot_be_license',)
('chatbot_be_question',)
('chatbot_be_questionanswer',)
('chatbot_be_reviewanswer',)
('chatbot_be_reviewer',)
('chatbot_be_scrapeddata',)
('chatbot_be_source',)
('django_admin_log',)
('django_content_type',)
('django_migrations',)
('django_session',)


In [24]:
# Parameterized query
query = f"SELECT * FROM {table_name} WHERE id = %s"
value = ("31",)  # A tuple with one element

cursor.execute(query, value)
rows = cursor.fetchall()

print(f"Filtered data from the table {table_name}:")
for row in rows:
    print(row)

Filtered data from the table chatbot_be_scrapeddata:
(31, 'https://en.wikipedia.org/wiki/Balance_of_payments', 'html', '<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Balance of payments - Wikipedia\n  </title>\n  <script>\n   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-t

In [25]:
def clean_answer(text):
    """
    Clean the text in the 'answer' column by:
    - Fixing missing spaces between words.
    - Removing irrelevant numbers or patterns.
    """
    # Add a space where a lowercase letter is directly followed by an uppercase letter
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)
    
    # Correct specific word join issues
    text = re.sub(r'inductionis', 'induction is', text)
    text = re.sub(r'Ingame', 'In game', text)
    
    # Remove irrelevant numbers (isolated numbers or patterns)
    text = re.sub(r'\b\d+\b', '', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [ ]:
# Extract questions and answers from the content column
cleaned_data = []

for row in rows:
    # Targeting the content column in the database
    html_content = row[3]  # Adjust index based on actual column position
    
    # Parsing the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract text from headings and paragraphs
    question = None
    for element in soup.find_all(['h1', 'h2', 'h3', 'p']):
        text = element.get_text(strip=True)
        if element.name.startswith('h'):  # Headings as questions
            question = f"What is {text.lower()}?" if not text.endswith('?') else text
        elif element.name == 'p':  # Paragraphs as answers
            answer = text
            if question and len(answer) > 10:  # Ensure non-trivial answers
                # Clean the answer text before saving
                cleaned_answer = clean_answer(answer)
                cleaned_data.append({"question": question, "answer": cleaned_answer})
                question = None  # Reset question after pairing

# Convert to a DataFrame
df = pd.DataFrame(cleaned_data)

# Save to Excel
#removed name from file name. May not work
output_excel_file = './DATA_SCIENCE/MIS 5000/QA/Balance-of-payment-QA.xlsx'
df.to_excel(output_excel_file, index=False, encoding='utf-8', sheet_name="Balance-of-payment QA Data")

print(f"QA dataset saved to {output_excel_file}")

QA dataset saved to C:/Users/sbah1/DATA_SCIENCE/MIS 5000/QA/Balance-of-payment-QA.xlsx


In [ ]:
# Close the connection
cursor.close()
conn.close()